In [1]:
import pandas as pd
import numpy as np
import os,sys
import re
pwd = os.getcwd()

In [2]:
# reading dataset
kb_df = pd.read_csv(pwd+"//kabitakitchen.csv", encoding_errors='ignore')
kb_df

,id,commentText,Labels
0,Ugy_CBm-_CKA3YqrzcB4AaABAg,Pudina ptta nhi dalu to,7
1,Ugy9mx9nuTWJu4dRac14AaABAg,Chiken kacha tu ni rhy ga sis,7
2,Ugz8T2MKLYucL3dM9nh4AaABAg,"Hello mam, I love your all recipes.... 😋😋😋\nAl...",4
3,Ugx_1cCjRbCaDgL0FLF4AaABAg,Its awesome recipe plzz make handi chicken in ...,2
4,UgzLhKVAJ6NN3nZXyjN4AaABAg,Yeh jo measurement hai.........kitne logon ke ...,7
...,...,...,...
4895,UgjFXyC0Qhzk5ngCoAEC,i love chole...thank you kabitaji for sharing ...,1
4896,UghP3bitlJuM13gCoAEC,thnakyou mm,1
4897,UghztLZOqvedfXgCoAEC,thanks mam,1
4898,UggX5Fi2Y430zXgCoAEC,u r fabulous,4


In [3]:
kb_df.shape

(4900, 3)

In [4]:
# replacing enter with ';'
kb_df['commentText'] = kb_df['commentText'].str.replace('\n',';')
kb_df

,id,commentText,Labels
0,Ugy_CBm-_CKA3YqrzcB4AaABAg,Pudina ptta nhi dalu to,7
1,Ugy9mx9nuTWJu4dRac14AaABAg,Chiken kacha tu ni rhy ga sis,7
2,Ugz8T2MKLYucL3dM9nh4AaABAg,"Hello mam, I love your all recipes.... 😋😋😋;All...",4
3,Ugx_1cCjRbCaDgL0FLF4AaABAg,Its awesome recipe plzz make handi chicken in ...,2
4,UgzLhKVAJ6NN3nZXyjN4AaABAg,Yeh jo measurement hai.........kitne logon ke ...,7
...,...,...,...
4895,UgjFXyC0Qhzk5ngCoAEC,i love chole...thank you kabitaji for sharing ...,1
4896,UghP3bitlJuM13gCoAEC,thnakyou mm,1
4897,UghztLZOqvedfXgCoAEC,thanks mam,1
4898,UggX5Fi2Y430zXgCoAEC,u r fabulous,4


In [5]:
# removing emojis
import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [6]:
# remove special characters 
import re

def remove_spechar(text):
    r = re.sub(r'\W+', ' ', text)
    return r.strip()

In [7]:
X = list(map(remove_emoji,list(kb_df['commentText'])))
X = list(map(remove_spechar,X))
y=list(kb_df['Labels'])

C:\Users\murth\AppData\Local\Temp/ipykernel_21844/1096523203.py:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', text)


In [8]:
import torch
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('impyadav/GPT2-FineTuned-Hinglish-Song-Generation')

In [9]:
print(X)

['Pudina ptta nhi dalu to', 'Chiken kacha tu ni rhy ga sis', 'Hello mam I love your all recipes All the ingredients are easily available and your way of explaining is too good', 'Its awesome recipe plzz make handi chicken in handi', 'Yeh jo measurement hai kitne logon ke liye hai', 'Kabita mam I tried ur egg biryani everyone in my house just loved it thank so much that was so delicious it was all because of ur recipe', 'cooker me kar sakte he na', 'Mujhe bhot ache lagi apki respi mene subscribe kardia bhot ache he', 'Mam dahi k jgh p kuch or use kr skte h kya', 'Wooooooo it s very yummmmmm I love it', 'This is a perfect biryani recipe Apko follow kar k banaya acchi bani biryani', 'Hi Didi I was always curious that How Biryani Made Thank you so much for putting this detailed video This Weekend I will try and serve it to family Really Motivatied', 'thanx respect from Madam appne tel nahy dala', 'I made this it taste awesome thank you kabita ji', 'You re amazing', 'nice video', 'Aur kya i

In [10]:
# Load pre-trained model (weights)
model = GPT2Model.from_pretrained('impyadav/GPT2-FineTuned-Hinglish-Song-Generation',
                                  output_hidden_states = True # Whether the model returns all hidden-states.
                                  )


# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
                                  
def gpt_vectorization(text):
    # Split the sentence into tokens.    
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs[2]
    token_vecs = hidden_states[-1][0]
    
    # Calculate the average of all the token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return np.array(sentence_embedding)

Some weights of the model checkpoint at impyadav/GPT2-FineTuned-Hinglish-Song-Generation were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
gpt_vectors = list(map(gpt_vectorization,X))

In [12]:
print(gpt_vectors)

[array([ 0.35951754, -0.1283862 ,  0.23764075, ..., -0.22191563,
       -0.13706535,  0.44153795], dtype=float32), array([ 0.3016493 , -0.13122067, -0.10731311, ..., -0.45049778,
       -0.50828576,  0.38644338], dtype=float32), array([ 0.30659828, -0.05666965,  0.02504899, ..., -0.22024027,
       -0.39821628,  0.18410228], dtype=float32), array([ 0.5958014 ,  0.08369368,  0.33161196, ..., -0.13332887,
       -0.5110125 , -0.17553176], dtype=float32), array([ 0.5087606 , -0.40880585, -0.15530702, ..., -0.41273385,
       -0.28015164,  0.48916638], dtype=float32), array([ 0.3725569 , -0.10798891,  0.06855595, ..., -0.34316537,
       -0.4252578 ,  0.31279802], dtype=float32), array([ 0.46627855, -0.22952767, -0.03019856, ..., -0.34694028,
       -0.60979927,  0.5488764 ], dtype=float32), array([ 0.27654493, -0.04053539,  0.04596764, ..., -0.3161163 ,
       -0.37874913,  0.47241232], dtype=float32), array([ 0.55476725, -0.05199949,  0.07759833, ..., -0.35323486,
       -0.5114805 ,  0.

In [13]:
gpt_df = pd.DataFrame(gpt_vectors)
gpt_df

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.359518,-0.128386,0.237641,-0.216649,0.024827,-0.140394,0.268709,-0.162048,-0.358191,-0.601336,...,0.644371,-0.346752,0.508427,0.083801,0.375445,0.365181,-0.106916,-0.221916,-0.137065,0.441538
1,0.301649,-0.131221,-0.107313,-0.078666,0.081474,-0.003283,0.288077,-0.143368,-0.152177,-0.647573,...,0.459304,-0.152174,0.403902,0.098448,0.218510,0.363727,0.054427,-0.450498,-0.508286,0.386443
2,0.306598,-0.056670,0.025049,0.238559,0.078060,-0.051240,0.234429,0.089972,0.184990,-0.627072,...,0.313090,0.100554,0.211385,-0.008834,0.004470,-0.002159,-0.184643,-0.220240,-0.398216,0.184102
3,0.595801,0.083694,0.331612,0.053873,-0.182684,-0.155576,0.257429,0.116807,0.060145,-0.680163,...,0.472685,0.274597,0.368737,-0.018716,-0.048141,0.320237,-0.211636,-0.133329,-0.511012,-0.175532
4,0.508761,-0.408806,-0.155307,-0.048241,-0.248964,-0.207166,0.435097,-0.023624,-0.041416,-0.587448,...,0.631569,-0.076014,0.471921,-0.175366,0.334395,0.535957,-0.176932,-0.412734,-0.280152,0.489166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,0.443883,0.072720,0.079788,-0.021844,0.023404,-0.153367,0.266912,0.077919,0.108239,-0.647003,...,0.546835,-0.059523,0.218219,0.123918,-0.102055,0.270535,-0.355384,0.108380,-0.591704,0.181753
4896,0.342469,-0.002747,0.595338,-0.240493,0.275101,-0.180448,0.081999,0.172812,-0.220215,-0.686709,...,0.254716,-0.215815,0.248326,0.117703,0.378491,0.273489,-0.203396,-0.578099,-0.181603,-0.020670
4897,0.332326,-0.219616,0.081273,-0.115134,0.248612,0.112681,0.548595,0.206297,-0.649741,-0.683706,...,-0.162408,-0.095459,0.044059,0.021094,0.208955,-0.044803,-0.568803,-0.582562,0.118590,0.475229
4898,0.114984,-0.312958,0.247082,-0.272925,-0.268314,0.182002,0.382998,0.481826,-0.176431,-0.668590,...,0.000810,-0.289050,0.635597,-0.035292,0.153935,0.254305,0.120492,-0.131040,-0.199762,-0.011966


In [14]:
gpt_df.to_csv(pwd+"//gpt_hinglish_finetuned_vectorized_kabita_dataset.csv",index=False)